#Sistemas complejos
##Nicolas Garzón Carvajal y Juan Felipe Alvarez
###Computación usando origami

In [2]:
# -*- coding: utf-8 -*-
import json
from collections import deque

# -----------------------------------------------------------------------------
# 1. MODELO DE DATOS
# -----------------------------------------------------------------------------

class Pleat:
    """Representa una 'tira de papel' con un nombre y una señal booleana.
    El valor puede ser True, False o None (indefinido).
    """
    def __init__(self, name: str, value: bool = None):
        self.name = name
        self.value = value

    def __repr__(self):
        return f"Pleat(name='{self.name}', value={self.value})"


class Gadget:
    """Clase base para un bloque lógico (compuerta).
    Define la interfaz común para todas las compuertas.
    """
    def __init__(self, name: str, inputs: list[Pleat], outputs: list[Pleat]):
        if not isinstance(name, str):
            raise TypeError("El nombre del gadget debe ser un string.")
        if not all(isinstance(p, Pleat) for p in inputs + outputs):
            raise TypeError("Las entradas/salidas deben ser instancias de Pleat.")

        self.name = name
        self.inputs = inputs
        self.outputs = outputs

        # Validaciones específicas de cada gadget (se definen en las subclases)
        self._validate_pleats()

    def _validate_pleats(self):
        raise NotImplementedError

    def evaluate(self):
        raise NotImplementedError

    def __repr__(self):
        return f"{self.__class__.__name__}(name='{self.name}')"


# -----------------------------------------------------------------------------
# 2. BIBLIOTECA MÍNIMA DE GADGETS (Compuertas Lógicas)
# -----------------------------------------------------------------------------

class NOTGadget(Gadget):
    def _validate_pleats(self):
        if len(self.inputs) != 1 or len(self.outputs) != 1:
            raise ValueError("NOTGadget debe tener 1 entrada y 1 salida.")

    def evaluate(self):
        in_val = self.inputs[0].value
        self.outputs[0].value = None if in_val is None else not in_val


class ANDGadget(Gadget):
    def _validate_pleats(self):
        if len(self.inputs) != 2 or len(self.outputs) != 1:
            raise ValueError("ANDGadget debe tener 2 entradas y 1 salida.")

    def evaluate(self):
        in1, in2 = self.inputs[0].value, self.inputs[1].value
        self.outputs[0].value = None if (in1 is None or in2 is None) else (in1 and in2)


class ORGadget(Gadget):
    def _validate_pleats(self):
        if len(self.inputs) != 2 or len(self.outputs) != 1:
            raise ValueError("ORGadget debe tener 2 entradas y 1 salida.")

    def evaluate(self):
        in1, in2 = self.inputs[0].value, self.inputs[1].value
        self.outputs[0].value = None if (in1 is None or in2 is None) else (in1 or in2)


class NANDGadget(Gadget):
    def _validate_pleats(self):
        if len(self.inputs) != 2 or len(self.outputs) != 1:
            raise ValueError("NANDGadget debe tener 2 entradas y 1 salida.")

    def evaluate(self):
        in1, in2 = self.inputs[0].value, self.inputs[1].value
        self.outputs[0].value = None if (in1 is None or in2 is None) else not (in1 and in2)


# -----------------------------------------------------------------------------
# 3. RED (NETWORK) Y LÓGICA DE PROPAGACIÓN
# -----------------------------------------------------------------------------

class Network:
    """Gestiona un conjunto de gadgets y pleats interconectados (DAG)."""

    GADGET_MAP = {
        "NOT": NOTGadget,
        "AND": ANDGadget,
        "OR": ORGadget,
        "NAND": NANDGadget,
    }

    def __init__(self):
        self.pleats = {}      # Diccionario: nombre -> Pleat
        self.gadgets = {}     # Diccionario: nombre -> Gadget
        self.propagation_log = []

    def add_pleat(self, name: str):
        if name not in self.pleats:
            self.pleats[name] = Pleat(name)
        return self.pleats[name]

    def add_gadget(self, gadget_type: str, name: str, input_names: list[str], output_names: list[str]):
        if name in self.gadgets:
            raise ValueError(f"Ya existe un gadget con el nombre '{name}'.")
        for pname in input_names + output_names:
            self.add_pleat(pname)

        inputs = [self.pleats[pname] for pname in input_names]
        outputs = [self.pleats[pname] for pname in output_names]

        gadget_class = self.GADGET_MAP.get(gadget_type.upper())
        if not gadget_class:
            raise ValueError(f"Tipo de gadget desconocido: {gadget_type}")

        self.gadgets[name] = gadget_class(name, inputs, outputs)
        return self.gadgets[name]

    def set_inputs(self, initial_values: dict):
        for name, value in initial_values.items():
            if name in self.pleats:
                self.pleats[name].value = value
            else:
                raise ValueError(f"Pleat no existente: '{name}'")

    def _get_topological_sort(self) -> list[Gadget]:
        in_degree = {name: 0 for name in self.gadgets}
        adj = {name: [] for name in self.gadgets}

        pleat_producers = {}
        for g_name, gadget in self.gadgets.items():
            for out_pleat in gadget.outputs:
                pleat_producers[out_pleat.name] = g_name

        for g_name, gadget in self.gadgets.items():
            for in_pleat in gadget.inputs:
                if in_pleat.name in pleat_producers:
                    producer = pleat_producers[in_pleat.name]
                    if producer != g_name:
                        adj[producer].append(g_name)
                        in_degree[g_name] += 1

        queue = deque([name for name, degree in in_degree.items() if degree == 0])
        sorted_order = []
        while queue:
            g_name = queue.popleft()
            sorted_order.append(self.gadgets[g_name])
            for neighbor in adj[g_name]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    queue.append(neighbor)

        if len(sorted_order) != len(self.gadgets):
            raise Exception("Ciclo de dependencias detectado en la red.")

        return sorted_order

    def run(self, log=False):
        self.propagation_log = []
        evaluation_order = self._get_topological_sort()
        if log:
            print("--- Iniciando Simulación ---")
            print(f"Orden de evaluación: {[g.name for g in evaluation_order]}\n")

        for gadget in evaluation_order:
            gadget.evaluate()
            if log:
                log_entry = {
                    "gadget": gadget.name,
                    "type": gadget.__class__.__name__,
                    "inputs": {p.name: p.value for p in gadget.inputs},
                    "outputs": {p.name: p.value for p in gadget.outputs}
                }
                self.propagation_log.append(log_entry)
                print(f"Evaluando '{gadget.name}': {log_entry['inputs']} -> {log_entry['outputs']}")

        if log:
            print("--- Simulación Finalizada ---\n")

    def get_pleat_values(self, pleat_names: list[str]) -> dict:
        return {name: self.pleats[name].value for name in pleat_names if name in self.pleats}

    # ------------------ NUEVO: Cargar desde JSON ------------------

    @classmethod
    def load_from_json(cls, path: str):
        """Construye una red a partir de un archivo JSON."""
        with open(path, "r", encoding="utf-8") as f:
            config = json.load(f)

        net = cls()

        for pleat_name in config.get("pleats", []):
            net.add_pleat(pleat_name)

        for g in config.get("gadgets", []):
            net.add_gadget(g["type"], g["name"], g.get("inputs", []), g.get("outputs", []))

        inputs = config.get("inputs", {})
        if inputs:
            net.set_inputs(inputs)

        return net, config.get("outputs", [])

    @classmethod
    def run_from_json(cls, path: str, log=False):
        net, outputs = cls.load_from_json(path)
        net.run(log=log)
        return net.get_pleat_values(outputs)


# -----------------------------------------------------------------------------
# 4. VALIDACIÓN Y PRUEBAS
# -----------------------------------------------------------------------------

def run_gadget_test(gadget_type: str, test_cases: list, expected_results: list):
    print(f"--- Probando {gadget_type.upper()}Gadget ---")
    has_failed = False
    for i, (inputs, expected) in enumerate(zip(test_cases, expected_results)):
        net = Network()
        input_pleat_names = [f'in{j+1}' for j in range(len(inputs))]
        net.add_gadget(gadget_type, 'test_gadget', input_pleat_names, ['out1'])
        net.set_inputs({name: val for name, val in zip(input_pleat_names, inputs)})
        net.run()
        result = net.get_pleat_values(['out1'])['out1']
        if result == expected:
            print(f"  Caso {i+1}: Entradas {inputs} -> Salida {result} (Éxito)")
        else:
            print(f"  Caso {i+1}: Entradas {inputs} -> Salida {result} (FALLÓ, esperado {expected})")
            has_failed = True
    print("Prueba finalizada." if not has_failed else "Prueba CON FALLOS.", "\n")


# --- Pruebas Unitarias ---

run_gadget_test('NOT', [[False], [True], [None]], [True, False, None])
run_gadget_test('AND', [[False, False], [False, True], [True, False], [True, True], [True, None]], [False, False, False, True, None])
run_gadget_test('OR', [[False, False], [False, True], [True, False], [True, True], [False, None]], [False, True, True, True, None])
run_gadget_test('NAND', [[False, False], [False, True], [True, False], [True, True], [None, True]], [True, True, True, False, None])


# --- Medio Sumador (Half-Adder) ---

def build_half_adder_network() -> Network:
    net = Network()
    net.add_pleat('a'); net.add_pleat('b'); net.add_pleat('sum'); net.add_pleat('carry')
    net.add_pleat('or_out'); net.add_pleat('nand_out')
    net.add_gadget('OR', 'or1', ['a', 'b'], ['or_out'])
    net.add_gadget('NAND', 'nand1', ['a', 'b'], ['nand_out'])
    net.add_gadget('AND', 'and_sum', ['or_out', 'nand_out'], ['sum'])
    net.add_gadget('AND', 'and_carry', ['a', 'b'], ['carry'])
    return net

def test_half_adder():
    print("--- Probando Medio Sumador ---")
    test_cases = [{'a': False, 'b': False}, {'a': False, 'b': True}, {'a': True, 'b': False}, {'a': True, 'b': True}]
    expected = [{'sum': False, 'carry': False}, {'sum': True, 'carry': False}, {'sum': True, 'carry': False}, {'sum': False, 'carry': True}]
    for i, inputs in enumerate(test_cases):
        ha_net = build_half_adder_network()
        ha_net.set_inputs(inputs)
        print(f"\nCaso {i+1}: a={int(inputs['a'])}, b={int(inputs['b'])}")
        ha_net.run(log=True)
        outputs = ha_net.get_pleat_values(['sum', 'carry'])
        print("Resultado:", outputs, "Esperado:", expected[i])

test_half_adder()


# -----------------------------------------------------------------------------
# 5. EJEMPLO: Cargar desde JSON
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    try:
        result = Network.run_from_json("half_adder.json", log=True)
        print("Resultado desde JSON:", result)
    except FileNotFoundError:
        print("⚠️ No se encontró 'half_adder.json'. Crea el archivo para probar carga desde JSON.")

--- Probando NOTGadget ---
  Caso 1: Entradas [False] -> Salida True (Éxito)
  Caso 2: Entradas [True] -> Salida False (Éxito)
  Caso 3: Entradas [None] -> Salida None (Éxito)
Prueba finalizada. 

--- Probando ANDGadget ---
  Caso 1: Entradas [False, False] -> Salida False (Éxito)
  Caso 2: Entradas [False, True] -> Salida False (Éxito)
  Caso 3: Entradas [True, False] -> Salida False (Éxito)
  Caso 4: Entradas [True, True] -> Salida True (Éxito)
  Caso 5: Entradas [True, None] -> Salida None (Éxito)
Prueba finalizada. 

--- Probando ORGadget ---
  Caso 1: Entradas [False, False] -> Salida False (Éxito)
  Caso 2: Entradas [False, True] -> Salida True (Éxito)
  Caso 3: Entradas [True, False] -> Salida True (Éxito)
  Caso 4: Entradas [True, True] -> Salida True (Éxito)
  Caso 5: Entradas [False, None] -> Salida None (Éxito)
Prueba finalizada. 

--- Probando NANDGadget ---
  Caso 1: Entradas [False, False] -> Salida True (Éxito)
  Caso 2: Entradas [False, True] -> Salida True (Éxito)
  Ca